### Imports

In [1]:
import pandas as pd
import seaborn as sns
import sys
import glob
import pathlib
from oak_cli.utils.logging import logger
from icecream import ic

from oak_cli.evaluation.graph_utils import PALETTE
from oak_cli.evaluation.addons.flops.graph_utils.keys import (
    RUN_ID_KEY,
    CPU_KEY,
    DISK_LAST_KEY,
    DISK_START_KEY,
    MEMORY_KEY,
    NETWORK_LAST_KEYS,
    NETWORK_START_KEYS,
    STAGE_KEY,
    TIME_START_KEY,
    ACCURACY_KEY,
    LOSS_KEY,
)
from oak_cli.evaluation.addons.flops.graph_utils.main import draw_graph
from oak_cli.evaluation.addons.flops.graph_utils.special_graphs import (
    draw_box_violin_plot_for_each_stage,
    draw_line_graph_with_all_runs,
    draw_trained_model_comparison_graph,
)
from oak_cli.evaluation.addons.flops.main import EvaluationRunFLOpsProjectStage

IndentationError: unexpected indent (main.py, line 8)

### Settings

In [ ]:
USE_MINUTES = True

### Constants

In [ ]:
#csv_dir = get_csv_dir_for_scenario(EvaluationScenario.FLOPS)
csv_dir = pathlib.Path("/home/alex/oakestra-cli/oak_cli/evaluation/addons/flops/csv/monolith_mnist_sklearn_small_without_baseimages")
#csv_dir = pathlib.Path("/home/alex/oakestra-cli/oak_cli/evaluation/addons/flops/csv/monolith_hierarchical_mnist_sklearn_small_without_baseimages")

### Data Loading

In [ ]:
csv_files = glob.glob(f'{csv_dir}/evaluation_run_*.csv')
df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Add a numerical stage ID (instead of the string) for future numerical manipulations.
STAGE_ID_KEY = "STAGE ID"
df[STAGE_ID_KEY]  =  df[STAGE_KEY].apply(lambda stage_name: EvaluationRunFLOpsProjectStage(stage_name).get_index())

#trained_model_df = pd.read_csv(TRAINED_MODEL_PERFORMANCE_CSV)
trained_model_df = pd.read_csv(csv_dir / "trained_models.csv")

# NOTE: The CSV "time-since-start" values are very precise, thus they differ (slightly) between Evaluation-Runs.
# This difference leads to issues when trying to plot them in an aggregated way.
# To fix this we cast the floats to ints instead. I.e. we are looking at whole seconds - which is fine for this concrete use-case.
df[[TIME_START_KEY]] = df[[TIME_START_KEY]].astype(int)

MEDIAN_RUNTIME__MINUTES = round(df.groupby(RUN_ID_KEY)[TIME_START_KEY].max().median() / 60, 2)

df.set_index(TIME_START_KEY, inplace=True)

if USE_MINUTES:
    df.index = df.index / 60

singular_run_df = df[df[RUN_ID_KEY] == (df[RUN_ID_KEY].max() // 2)]

### Error Handing & Checking

In [ ]:
if not csv_dir.exists():
    logger.error(f"{csv_dir} does not exist yet!")
    sys.exit(1)

### Common Graph Styling

In [ ]:
sns.set_style("whitegrid")

---

# Graphs Drawing

### CPU & Memory

In [ ]:
draw_graph(
    title="Singular Example Evaluation Run",
    data=singular_run_df[[CPU_KEY, MEMORY_KEY, STAGE_KEY]],
    show_stages=True,
    use_percentage_limits=True,
)

In [ ]:
draw_graph(
    title="Evaluation Runs Average",
    data=df[[CPU_KEY, MEMORY_KEY, STAGE_KEY]],
    use_percentage_limits=True,
    #show_stages=True,
    #stages_color_intensity=0.01,
)

In [ ]:
sns.ecdfplot(
    #x=STAGE_KEY,
    #x=CPU_KEY,
    data=df[[CPU_KEY, MEMORY_KEY]],
    palette=PALETTE,
    #hue=STAGE_KEY,
)
sns.kdeplot(
    #x=STAGE_KEY,
    #x=CPU_KEY,
    data=df[[CPU_KEY, MEMORY_KEY]],
    palette=PALETTE,
    #hue=STAGE_KEY,
    cumulative=True,
)

In [ ]:
draw_line_graph_with_all_runs(
    data=df,
    y_label="CPU Usage (%)",
    key=CPU_KEY,
    )

In [ ]:
draw_line_graph_with_all_runs(
    data=df,
    y_label="Memory Usage (%)",
    key=MEMORY_KEY,
    )

In [ ]:
draw_box_violin_plot_for_each_stage(
    data=df,
    key=CPU_KEY,
    y_label="CPU Usage (%)",
    y_lim=(0,100),
)

In [ ]:
draw_box_violin_plot_for_each_stage(
    data=df,
    key=MEMORY_KEY,
    y_label="Memory Usage (%)",
    y_lim=(40,80)
)

In [ ]:
# Step 1 & 2: Group by 'experiment_run' and 'stage', then sum 'time'
#grouped = df.groupby([RUN_ID_KEY, STAGE_KEY])[TIME_START_KEY].sum().reset_index()

# _df = df.copy()

# # _df["STAGE ID"]  =  _df[STAGE_KEY].apply(lambda stage_name: EvaluationRunFLOpsProjectStage(stage_name).get_index())
# #_df
# #grouped = _df.groupby(STAGE_ID_KEY)[TIME_START_KEY].sum().reset_index()
# _df.sort_values(by=[RUN_ID_KEY, STAGE_ID_KEY], inplace=True)
# grouped = _df.groupby([RUN_ID_KEY, STAGE_ID_KEY]).size().reset_index(name="count")
# max_count_per_stage = grouped.groupby(STAGE_ID_KEY)['count'].transform(max)
# normalized_counts = grouped['count'] / max_count_per_stage * 100
# type(normalized_counts)
# # average_percentages = normalized_counts.groupby(STAGE_ID_KEY).mean().round(2)
# # average_percentages



In [ ]:
# _df[[TIME_START_KEY]] = _df[[TIME_START_KEY]].astype(int)
# MEDIAN_RUNTIME__MINUTES
###########################

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

_df = df.copy()
_df.reset_index(inplace=True)
#_df[[TIME_START_KEY]] = (_df[[TIME_START_KEY]] * 60).astype(int)
_df[[TIME_START_KEY]] = round(_df[[TIME_START_KEY]],2)
grouped_by_stage_id_and_run_id_for_time_start = _df.groupby([STAGE_KEY, RUN_ID_KEY])[TIME_START_KEY]
stage_start_times = grouped_by_stage_id_and_run_id_for_time_start.min()
stage_end_times = grouped_by_stage_id_and_run_id_for_time_start.max()
stage_runtimes = stage_end_times - stage_start_times
#stage_runtimes = (stage_end_times - stage_start_times).to_frame()
#stage_runtimes.columns
#stage_runtimes[[STAGE_ID_KEY]]
#stage_runtimes


# _df.reset_index(inplace=True)
# 
# #average_evaluation_runtime = _df.gro [TIME_START_KEY].a
# #res = _df.groupby(RUN_ID_KEY)[TIME_START_KEY].agg()
# res = _df.groupby(RUN_ID_KEY)[TIME_START_KEY].max().median()

# res

#sns.lineplot(data=stage_runtimes)

# Create a new DataFrame to hold the final result
final_df = pd.DataFrame({
    STAGE_KEY: stage_start_times.index.get_level_values(STAGE_KEY),
    RUN_ID_KEY: stage_start_times.index.get_level_values(RUN_ID_KEY),
    'RUNTIME': stage_runtimes.values
})

# Set TIME_START_KEY as the index if needed
#final_df.set_index(TIME_START_KEY, inplace=True)
#final_df
#print(final_df)

draw_graph(
    data=final_df,
    plot_functions=[
        lambda: sns.barplot(
            data=final_df,
            y=STAGE_KEY,
            x="RUNTIME",
            #palette=PALETTE,
        )
    ], 
)

# sns.barplot(
#             data=final_df,
#             y=STAGE_KEY,
#             x="RUNTIME",
#             palette=PALETTE,
#         )

# sns.barplot(
#             data=df[[CPU_KEY]],
#             # y=STAGE_KEY,
#             # x="RUNTIME",
#             # palette=PALETTE,
#         )

### Disk Space Changes

In [ ]:
_singular_run_df = singular_run_df.copy()
_singular_run_df[[DISK_START_KEY]] = _singular_run_df[[DISK_START_KEY]] / 1024

draw_graph(
    title="Singular Example Evaluation Run",
    data=_singular_run_df[[DISK_START_KEY, STAGE_KEY]],
    y_label="Disk Space Change (GB)",
    x_lim=(0, max(_singular_run_df.index)),
    y_lim=0,
    show_stages=True,
    
)

In [ ]:
_df = df.copy()
_df[[DISK_START_KEY]] = _df[[DISK_START_KEY]] / 1024

draw_graph(
    title="Evaluation Runs Average",
    data=_df[[DISK_START_KEY, STAGE_KEY]],
    y_label="Disk Space Change (GB)",
    x_lim=(0, max(_df.index)),
    #show_stages=True,
    #stages_color_intensity=0.01,
)

In [ ]:

max_y = singular_run_df[DISK_LAST_KEY].max()
draw_graph(
    title="Singular Example Evaluation Run",
    data=singular_run_df[[DISK_LAST_KEY, STAGE_KEY]],
    y_label="Disk Space Change Between Measurements (MB)",
    x_lim=(0, max(singular_run_df.index)),
    show_stages=True,
    stages_color_height=max_y,
    y_lim=(0, max_y)
)

In [ ]:
max_y = df[DISK_LAST_KEY].max()
draw_graph(
    title="Evaluation Runs Average",
    data=df[[DISK_LAST_KEY, STAGE_KEY]],
    y_label="Disk Space Change Between Measurements (MB)",
    x_lim=(0, max(df.index)),
    #y_lim=(0, max_y),
    y_lim=(0, 2000),
    #show_stages=True,
    #stages_color_intensity=0.01,
)

### Network

In [ ]:
_singular_run_df = singular_run_df.copy()
_singular_run_df[NETWORK_START_KEYS] = _singular_run_df[NETWORK_START_KEYS] / 1024

draw_graph(
    title="Evaluation Runs Average",
    data=_singular_run_df[NETWORK_START_KEYS + [STAGE_KEY]],
    y_label="Network Change (GB)",
    x_lim=(0, max(_singular_run_df.index)),
    show_stages=True,
)

In [ ]:
_df = df.copy()
_df[NETWORK_START_KEYS] = _df[NETWORK_START_KEYS] / 1024

draw_graph(
    title="Evaluation Runs Average",
    data=_df[NETWORK_START_KEYS + [STAGE_KEY]],
    y_label="Network Change (GB)",
    x_lim=(0, max(_df.index)),
    #show_stages=True,
    #stages_color_intensity=0.01,
)

In [ ]:
draw_graph(
    title="Evaluation Runs Average",
    data=singular_run_df[NETWORK_LAST_KEYS + [STAGE_KEY]],
    y_label="Network Changes Between Measurements (MB)",
    x_lim=(0, max(singular_run_df.index)),
    y_lim=(0, 1000),
    show_stages=True,
    stages_color_height=1000
)

In [ ]:
draw_graph(
    title="Evaluation Runs Average",
    data=df[NETWORK_LAST_KEYS + [STAGE_KEY]],
    y_label="Network Changes Between Measurements (MB)",
    x_lim=(0, max(df.index)),
    y_lim=(0, 1000),
    #show_stages=True,
    #stages_color_intensity=0.01,
)

### Trained Models

In [ ]:
draw_trained_model_comparison_graph(
    data=trained_model_df,
    key=ACCURACY_KEY,
    y_label="Accuracy (%)"
)

In [ ]:
draw_trained_model_comparison_graph(
    data=trained_model_df,
    key=LOSS_KEY,
    y_label="Loss (%)"
)